In [19]:
import numpy as np
import math
from itertools import product
import random
#computes the entropy of a probability distribution given a list of probabilities "probabilities"
def H(probabilities):
    sum = 0.0
    for i in probabilities:
        if (i <= 0):
            sum += 0
        else:
            sum -= i * math.log2(i)
    return sum

In [3]:
#computes the objective function given numpy arrays p, a, b
def loss(p, a, b):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += p[i] * (H([a[i], 1 - a[i]]) + H([b[i], 1 - b[i]]))
    return sum

In [4]:
def h(x):
    return H([x, 1-x])

In [5]:
def L(x):
    return h(x) + 1 - x

In [6]:
def LHS(p, a, b, c):
    n = len(p)
    sum = 0
    for i in range(n):
        sum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
    if (sum < 0 or sum > 1):
        return False
    return L(sum)

In [7]:
def RHS(p, a, b, c):
    entropy_sum = 0
    n = len(p)
#     print(a)
#     print(b)
#     print(c)
    for i in range(n):
        if ((a[i] * b[i] - c[i]) < 0 or (a[i] * (1 - b[i]) + c[i]) < 0 or (b[i] * (1 - a[i]) + c[i]) < 0 or ((1 - a[i]) * (1 - b[i]) + c[i]) < 0):
            return False

    for i in range(n):
        entropy = H([a[i] * b[i] - c[i], a[i] * (1 - b[i]) + c[i], (1 - a[i]) * b[i] + c[i], (1 - a[i]) * (1 - b[i]) - c[i]])
        entropy_sum += p[i] * entropy
    return entropy_sum

In [8]:
def LHS_minus_RHS(p, a, b, c):
    if (LHS(p,a,b,c) == False or RHS(p,a,b,c) == False):
        return False
    return LHS(p, a, b, c) - RHS(p, a, b, c)

In [9]:
def worst_c(p, a, b, alpha, iterations):
    n = len(p)
    c = np.zeros(n, dtype = float)
    prevc = np.zeros(n, dtype = float)
    for i in range(iterations):
        prevc[:] = c
        for j in range(n):
            cplus = list(c)
            cplus[j] += 0.0001
            cminus = list(c)
            cminus[j] -=  0.0001
            dxdc = (LHS_minus_RHS(p, a, b, cplus) - LHS_minus_RHS(p, a, b, cminus))/.0002
            c[j] -= dxdc * alpha
        for j in range(n):
            if (a[j] * b[j] - c[j] < 0 or a[j] * (1 - b[j]) + c[j] < 0 or b[j] * (1 - a[j]) + c[j] < 0 or (1 - a[j]) * (1 - b[j]) + c[j] < 0):
                c[:] = prevc
                return c
    return c

In [10]:
def L_minus_sum_RHS(p, a, b, alpha, iterations):
    c = worst_c(p, a, b, alpha, iterations)
    return LHS(p, a, b, c) - RHS(p, a, b, c)

In [21]:
%%time
#n = 1


def grid_search_U_val(u_val):
    optimal_rate = -1
    abest = [0]
    bbest = [0]
    counter = 0
    for i in range(100000000):
        if i % 10000000 == 0:
            print(i, optimal_rate, abest, bbest)
        a1 = (i % 10000)/10000
        b1 = (i // 10000)/10000
        if a1 == 0.5 or b1 == 0.5 or a1 + b1 == 1:
            continue
        a = [a1]
        b = [b1]
        X = binarySearchX([1], a, b, 30)
        c = find_c_dim_one(X, a[0], b[0])
        if LHS_minus_RHS([1], a, b, [c]) < -0.0001 or not LHS_minus_RHS([1], a, b, [c]):
            continue
        counter += 1
        rate = u_val * h(a1) + (1 - u_val) * h(b1)
        if rate > optimal_rate:
            optimal_rate = rate
            abest = a
            bbest = b
    return (optimal_rate, abest, bbest)

print(grid_search_U_val(0.1))
        

0 -1 [0] [0]
10000000 0.5217907904588079 [0.4917] [0.0999]
20000000 0.7350866965946008 [0.2799] [0.1999]
30000000 0.8623332979063936 [0.1859] [0.2999]
40000000 0.9305527442501372 [0.1337] [0.3999]
50000000 0.9470329265310393 [0.1037] [0.4801]
60000000 0.9470329265310393 [0.1037] [0.4801]
70000000 0.9470329265310393 [0.1037] [0.4801]
80000000 0.9470329265310393 [0.1037] [0.4801]
90000000 0.9470329265310393 [0.1037] [0.4801]
(0.9470329265310393, [0.1037], [0.4801])
CPU times: user 1h 13min 17s, sys: 12.4 s, total: 1h 13min 29s
Wall time: 1h 13min 39s


In [26]:
print(grid_search_U_val(0.2))

0 -1 [0] [0]
10000000 0.5749030600019637 [0.4917] [0.0999]
20000000 0.7484453433916964 [0.2799] [0.1999]
30000000 0.8434979701762917 [0.1859] [0.2999]
40000000 0.8902204561049427 [0.1338] [0.3998]
50000000 0.8967352870262564 [0.1126] [0.4544]
60000000 0.8967352870262564 [0.1126] [0.4544]
70000000 0.8967352870262564 [0.1126] [0.4544]
80000000 0.8967352870262564 [0.1126] [0.4544]
90000000 0.8967352870262564 [0.1126] [0.4544]
(0.8967352870262564, [0.1126], [0.4544])


In [28]:
print(grid_search_U_val(0.3))

0 -1 [0] [0]
10000000 0.6280153295451192 [0.4917] [0.0999]
20000000 0.761803990188792 [0.2799] [0.1999]
30000000 0.8246626424461895 [0.1859] [0.2999]
40000000 0.8499139432957865 [0.1338] [0.3998]
50000000 0.8505517754027353 [0.1263] [0.4181]
60000000 0.8505517754027353 [0.1263] [0.4181]
70000000 0.8505517754027353 [0.1263] [0.4181]
80000000 0.8505517754027353 [0.1263] [0.4181]
90000000 0.8505517754027353 [0.1263] [0.4181]
(0.8505517754027353, [0.1263], [0.4181])


In [ ]:
X = binarySearchX([1], abest, bbest, 30)
c = find_c_dim_one(X, abest[0], bbest[0])
print(c)

In [ ]:
worst_c([1], abest, bbest, 0.01, 1000)

In [ ]:
a1 = 0.625
b1 = 0.375
X = binarySearchX([1], [a1], [b1], 30)
c = find_c_dim_one(X, a1, b1)
LHS_minus_RHS([1], [a1], [b1], [c])

In [25]:
%%time
#n = 2

u_val = 0.1


def random_search_u_val(u_val):
    optimal_rate = -1
    abest = [0,0]
    bbest = [0,0]
    pbest = 0
    for i in range(100000000):
        if i % 10000000 == 0:
            print(i, optimal_rate, abest, bbest)
        a1 = random.randint(1, 999)/1000
        a2 = random.randint(1, 999)/1000
        b1 = random.randint(1, 999)/1000
        b2 = random.randint(1, 999)/1000
        p = random.randint(1, 999)/1000
        a = [a1, a2]
        b = [b1, b2]
        if a1 == 0.5 or b1 == 0.5 or a1 + b1 == 1 or a2 == 0.5 or b2 == 0.5 or a2 + b2 == 1:
            continue
        X = binarySearchX([p, 1 - p], a, b, 30)
        c = [find_c_dim_one(X, a[0], b[0]), find_c_dim_one(X, a[1], b[1])]
        if LHS_minus_RHS([p, 1 - p], a, b, c) < -0.0001 or not LHS_minus_RHS([p, 1 - p], a, b, c):
            continue
        rate = p * (u_val * h(a1) + (1 - u_val) * h(b1)) + (1 - p) * (u_val * h(a2) + (1 - u_val) * h(b2))
        if rate > optimal_rate:
            optimal_rate = rate
            abest =  a
            bbest = b
            pbest = p
    return (optimal_rate)

print(random_search_u_val(0.1))

0 -1 [0, 0] [0, 0]
10000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
20000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
30000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
40000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
50000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
60000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
70000000 0.9469267569534705 [0.9, 0.752] [0.517, 0.518]
80000000 0.9469429153202701 [0.104, 0.027] [0.483, 0.452]
90000000 0.9469583641882975 [0.132, 0.914] [0.477, 0.519]
0.9469583641882975
CPU times: user 2h 14min, sys: 17.9 s, total: 2h 14min 18s
Wall time: 2h 16min 15s


In [33]:
print((0.132 + (1 - 0.914))/2)
print((0.477 + (1 - 0.519))/2)

0.10899999999999999
0.479


In [27]:
print(random_search_u_val(0.2))

0 -1 [0, 0] [0, 0]
10000000 0.8964765230942042 [0.114, 0.231] [0.447, 0.434]
20000000 0.8965033582478726 [0.078, 0.883] [0.444, 0.537]
30000000 0.8965728942972436 [0.891, 0.833] [0.541, 0.555]
40000000 0.8966015470064223 [0.907, 0.875] [0.549, 0.545]
50000000 0.8966063659558093 [0.121, 0.077] [0.461, 0.445]
60000000 0.8966063659558093 [0.121, 0.077] [0.461, 0.445]
70000000 0.8966063659558093 [0.121, 0.077] [0.461, 0.445]
80000000 0.8966497932752797 [0.884, 0.908] [0.552, 0.547]
90000000 0.8966497932752797 [0.884, 0.908] [0.552, 0.547]
0.8966497932752797


In [34]:
print((0.884 + 0.908)/2)
print((0.552 + 0.547)/2)

0.896
0.5495000000000001


In [29]:
print(random_search_u_val(0.3))

0 -1 [0, 0] [0, 0]
10000000 0.8503286281510652 [0.756, 0.877] [0.501, 0.58]
20000000 0.8504463280519458 [0.107, 0.871] [0.425, 0.588]
30000000 0.8504463280519458 [0.107, 0.871] [0.425, 0.588]
40000000 0.8504774347137134 [0.126, 0.875] [0.51, 0.579]
50000000 0.8504774347137134 [0.126, 0.875] [0.51, 0.579]
60000000 0.8505206113307254 [0.873, 0.37] [0.584, 0.359]
70000000 0.8505206113307254 [0.873, 0.37] [0.584, 0.359]
80000000 0.8505206113307254 [0.873, 0.37] [0.584, 0.359]
90000000 0.8505206113307254 [0.873, 0.37] [0.584, 0.359]
0.8505206113307254


In [35]:
print((0.37 + (1 - 0.873))/2)
print((0.359 + (1 - 0.584)/2))

0.2485
0.567


In [12]:
def find_c_dim_one(x,a,b):
    p1 = 2*a*b-a-b-k(x)
    p2 = k(x)**2 + 2*(a+b-2*a*b)*k(x)+(a-b)**2
    if p2<0:
        return 0
    if x<1/3:
        return (p1+p2**.5)/2
    return (p1-p2**.5)/2

def k(x):
    return -4*x**2/((3*x-1)*(x+1))

def f3(x,p,a,b):
    s = 0
    for i in range(len(p)):
        s+=p[i]*(a[i]*(1-b[i])+(1-a[i])*b[i]+2*find_c_dim_one(x,a[i],b[i]))
    return s-x

def binarySearchX(p,a,b,steps):
    checking = 1/2
    stepSize = 1/4
    for i in range(steps):
        if f3(checking,p,a,b)>0:
            checking = checking+stepSize
        else:
            checking = checking-stepSize
        stepSize=stepSize/2
    return checking